<a href="https://colab.research.google.com/github/NeelanshaAgarwal/Face-Mask-Detection/blob/main/1facemaskexe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
!unzip -uq "/content/drive/My Drive/archive" -d "/content/drive/My Drive/archive"

In [ ]:
import os
main_dir = "/content/drive/MyDrive/archive/New Masks Dataset"
train_dir = os.path.join(main_dir,'Train')
test_dir = os.path.join(main_dir,'Test')
valid_dir = os.path.join(main_dir,'Validation')

train_mask_dir = os.path.join(train_dir,'Mask')
train_nomask_dir = os.path.join(train_dir,'Non Mask')

In [ ]:
print(train_dir)

/content/drive/MyDrive/archive/New Masks Dataset/Train


In [ ]:
train_mask_names = os.listdir(train_mask_dir)
print(train_mask_names[:10])

train_nomask_names = os.listdir(train_nomask_dir)
print(train_mask_names[:10])

['0037.jpg', '0042.jpg', '0029.jpg', '0027.jpg', '0018.jpg', '0040.jpg', '0022.jpg', '0006.jpg', '0019.jpg', '0003.jpg']
['0037.jpg', '0042.jpg', '0029.jpg', '0027.jpg', '0018.jpg', '0040.jpg', '0022.jpg', '0006.jpg', '0019.jpg', '0003.jpg']


In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   zoom_range = 0.2,
                                   rotation_range = 40,
                                   horizontal_flip = True
                                   )
test_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                   target_size = (150,150),
                                                   batch_size = 32,
                                                   class_mode = 'binary')
test_generator = test_datagen.flow_from_directory(test_dir,
                                                   target_size = (150,150),
                                                   batch_size = 32,
                                                   class_mode = 'binary')
valid_generator = validation_datagen.flow_from_directory(valid_dir,
                                                   target_size = (150,150),
                                                   batch_size = 32,
                                                   class_mode = 'binary')


Found 600 images belonging to 2 classes.
Found 100 images belonging to 2 classes.
Found 306 images belonging to 2 classes.


In [ ]:
train_generator.class_indices

{'Mask': 0, 'Non Mask': 1}

In [ ]:
train_generator.image_shape

(150, 150, 3)

In [ ]:
model = Sequential()
model.add(Conv2D(32,(3,3),padding='SAME',activation='relu',input_shape=(150,150,3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))

model.add(Conv2D(64,(3,3),padding='SAME',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))

model.add(Flatten())

model.add(Dense(256,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))
model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 150, 150, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 75, 75, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 75, 75, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 75, 75, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 37, 37, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 37, 37, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 87616)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 256)                 │      22,429,952 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             257 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 22,449,601 (85.64 MB)

 Trainable params: 22,449,601 (85.64 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.compile(Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history=model.fit(train_generator,
                  epochs = 30,
                  validation_data = valid_generator)

Epoch 1/30
19/19 ━━━━━━━━━━━━━━━━━━━━ 193s 9s/step - accuracy: 0.5564 - loss: 9.6930 - val_accuracy: 0.5000 - val_loss: 0.6932
Epoch 2/30
19/19 ━━━━━━━━━━━━━━━━━━━━ 64s 3s/step - accuracy: 0.5583 - loss: 0.6812 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 3/30
19/19 ━━━━━━━━━━━━━━━━━━━━ 81s 3s/step - accuracy: 0.5571 - loss: 0.6829 - val_accuracy: 0.4935 - val_loss: 0.6909
Epoch 4/30
19/19 ━━━━━━━━━━━━━━━━━━━━ 81s 3s/step - accuracy: 0.6963 - loss: 0.5756 - val_accuracy: 0.8268 - val_loss: 0.5163
Epoch 5/30
19/19 ━━━━━━━━━━━━━━━━━━━━ 83s 3s/step - accuracy: 0.8438 - loss: 0.4023 - val_accuracy: 0.8007 - val_loss: 0.5424
Epoch 6/30
19/19 ━━━━━━━━━━━━━━━━━━━━ 91s 3s/step - accuracy: 0.8773 - loss: 0.3505 - val_accuracy: 0.8333 - val_loss: 0.4576
Epoch 7/30
19/19 ━━━━━━━━━━━━━━━━━━━━ 74s 3s/step - accuracy: 0.8806 - loss: 0.3105 - val_accuracy: 0.8170 - val_loss: 0.5165
Epoch 8/30
19/19 ━━━━━━━━━━━━━━━━━━━━ 80s 3s/step - accuracy: 0.8944 - loss: 0.3238 - val_accuracy: 0.9020 - val_loss

In [ ]:
test_loss, test_acc = model.evaluate(test_generator)
print('test acc: {} test loss: {}'.format(test_acc,test_loss))

4/4 ━━━━━━━━━━━━━━━━━━━━ 13s 4s/step - accuracy: 0.8730 - loss: 0.2244
test acc: 0.8700000047683716 test loss: 0.22630569338798523


In [ ]:
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()
for f in uploaded.keys():
  img_path = '/content/'+f
  img = image.load_img(img_path, target_size=(150,150))
  images = image.img_to_array(img)
  images = np.expand_dims(images, axis=0)
  prediction = model.predict(images)
  if prediction == 0:
    print(f,'mask is there')
  else:
    print(f,'mask is not there')

Saving vecteezy_ai-generated-indian-female-student_39334804.jpg to vecteezy_ai-generated-indian-female-student_39334804.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
vecteezy_ai-generated-indian-female-student_39334804.jpg mask is not there


In [39]:
model.save('newsaved_model.h5')